# Import Event Log

In [55]:
import pandas as pd
import numpy as np
import pm4py
from pm4py.objects.conversion.log import converter as log_converter


if __name__ == "__main__":
    # Read the CSV file
    dataframe_log = pd.read_csv('../../data/logs/mobis.csv', sep=',')  

    # Drop the first column without knowing its name
    dataframe_log = dataframe_log.drop(dataframe_log.columns[0], axis=1)

    # Format the dataframe
    dataframe_log = pm4py.format_dataframe(
        dataframe_log, 
        case_id='case', 
        activity_key='activity', 
        timestamp_key='start'
    )

    # Convert the dataframe to event log
    log = log_converter.apply(dataframe_log)

/var/folders/s_/ch_w_j2d0sqf6dbdc0_224m40000gq/T/ipykernel_12704/539992039.py:15: DeprecatedWarning: format_dataframe is deprecated as of 2.3.0 and will be removed in 3.0.0. the format_dataframe function does not need application anymore.
  dataframe_log = pm4py.format_dataframe(


# Drop unnessary columns

In [56]:
dataframe_log = dataframe_log.drop(columns=['case'])
dataframe_log = dataframe_log.drop(columns=['cost'])
dataframe_log = dataframe_log.drop(columns=['case:concept:name'])
dataframe_log = dataframe_log.drop(columns=['concept:name'])
dataframe_log = dataframe_log.drop(columns=['@@index'])
dataframe_log = dataframe_log.drop(columns=['travel_start'])
dataframe_log = dataframe_log.drop(columns=['travel_end'])
dataframe_log = dataframe_log.drop(columns=['start'])
dataframe_log = dataframe_log.drop(columns=['end'])

# Preprocess

In [57]:
from sklearn.preprocessing import StandardScaler

# Convert to datetime format
dataframe_log['time:timestamp'] = pd.to_datetime(dataframe_log['time:timestamp'])

# Calculate elapsed time since the start of each case
dataframe_log['start_time'] = dataframe_log.groupby('@@case_index')['time:timestamp'].transform('min')
dataframe_log['elapsed_time'] = (dataframe_log['time:timestamp'] - dataframe_log['start_time']).dt.total_seconds()

# Normalize the elapsed time in minutes
scaler = StandardScaler()
dataframe_log['standardized_elapsed_time'] = scaler.fit_transform(dataframe_log[['elapsed_time']])

dataframe_log = dataframe_log.drop(columns=['start_time'])
dataframe_log = dataframe_log.drop(columns=['elapsed_time'])
dataframe_log = dataframe_log.drop(columns=['time:timestamp'])

In [58]:
codes, uniques = pd.factorize(dataframe_log['activity'])
dataframe_log['activity'] = codes + 1

In [59]:
# Fill NaN values with a placeholder before factorization
dataframe_log['type'].fillna('missing', inplace=True)

# Factorize the 'type' column
codes, uniques = pd.factorize(dataframe_log['type'])
dataframe_log['type'] = codes + 1

In [60]:
# Fill NaN values with a placeholder before factorization
dataframe_log['user'].fillna('missing', inplace=True)

# Factorize the 'type' column
codes, uniques = pd.factorize(dataframe_log['user'])
dataframe_log['user'] = codes + 1

In [61]:
dataframe_log

,activity,type,user,@@case_index,standardized_elapsed_time
0,1,1,1,0,-1.160944
1,2,1,1,0,-1.160857
2,3,1,1,0,-1.160842
3,4,2,2,0,-1.160842
4,5,1,1,0,-1.144594
...,...,...,...,...,...
55804,8,1,333,3353,-0.742182
55805,9,3,53,3353,-0.736776
55806,10,1,333,3353,-0.583572
55807,11,4,59,3353,-0.404036


# Generate Prefixes

In [62]:
df_activity = dataframe_log[['activity', '@@case_index']]
df_type = dataframe_log[['type', '@@case_index']]
df_user = dataframe_log[['user', '@@case_index']]
df_timestamp = dataframe_log[['standardized_elapsed_time', '@@case_index']]

In [63]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_prefix_windows(df, case_id_column='@@case_index', max_len=None):
    windows = []
    targets = []
    case_indices = []
    
    for case_id in df[case_id_column].unique():
        case_data = df[df[case_id_column] == case_id].drop(columns=[case_id_column]).to_numpy()
        
        # Optional: Make sure to sort the case data if there's an implicit order (e.g., by timestamps)
        # case_data = case_data.sort_values(by='timestamp_column').to_numpy()  # Uncomment and adjust if needed
        
        for i in range(1, len(case_data)):
            window = case_data[:i]
            target = case_data[i]
            windows.append(window)
            targets.append(target)
            case_indices.append(case_id)
    
    if max_len is None:
        max_len = max(len(window) for window in windows)
    
    # Pad sequences
    windows_padded = pad_sequences(windows, maxlen=max_len, padding='post', dtype='float32')
    
    # Convert targets to numpy array
    targets_array = np.array(targets, dtype='float32')
    case_indices_array = np.array(case_indices)
    
    return windows_padded, targets_array, case_indices_array

In [64]:
windows_activity, targets_activity, case_indices = generate_prefix_windows(df_activity)
windows_type, targets_type, case_indices = generate_prefix_windows(df_type)
windows_user, targets_user, case_indices = generate_prefix_windows(df_user)
windows_timestamp, targets_timestamp, case_indices = generate_prefix_windows(df_timestamp)

# GRU

### Architecture

- Separate Inputs for Each Attribute
- Each attribute is passed through an embedding layer
- Each attribute has its corresponding GRU encoder
- Selective Concatenation: After encoding, the outputs of these GRU layers are concatenated. However, this concatenation is selective, meaning it is structured in a way that prepares the data for effective synthesis without leaking information from the future (next event attributes)
- Decoder GRUs: Integrated Decoding: Post-concatenation, the combined attributes are processed through decoder GRU layers. These layers are tasked with integrating the data from different attributes and preparing it for final prediction. This step is where BINet v3 distinguishes itself by effectively using the interdependencies between different attributes to enhance prediction accuracy.
- Output Layer: Softmax Output for Each Attribute: For each attribute of the next event, a softmax layer predicts a probability distribution over all possible values. This allows the model to output the most likely next event and its attributes based on the learned dependencies and the history encoded by the GRUs.
- E: maximum case length
- We train BINet with a GRU size of 2E (two times the maximum case length)
- on mini batches of size 500 for 20 epochs

In [65]:
# Group by the @@case_index column and count the rows in each group
case_lengths = dataframe_log.groupby('@@case_index').size()

# Find the maximum value among the case lengths
E = case_lengths.max()

In [66]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Embedding, Dense, Dropout, Concatenate, BatchNormalization

def create_binetv3(num_activities, num_types, num_users, embedding_dim, gru_units, dropout_rate):
    # Input layers for each attribute
    input_activity = Input(shape=(None,), name='activity_input')
    input_type = Input(shape=(None,), name='type_input')
    input_user = Input(shape=(None,), name='user_input')
    input_timestamp = Input(shape=(None, 1), name='timestamp_input')  # Continuous input

    # Embedding layers for categorical attributes
    embedding_activity = Embedding(input_dim=num_activities + 1, output_dim=embedding_dim, mask_zero=True, name='activity_embedding')(input_activity)
    embedding_type = Embedding(input_dim=num_types + 1, output_dim=embedding_dim, mask_zero=True, name='type_embedding')(input_type)
    embedding_user = Embedding(input_dim=num_users + 1, output_dim=embedding_dim, mask_zero=True, name='user_embedding')(input_user)

    # Encoder GRUs with Batch Normalization for categorical attributes
    encoded_activity = GRU(units=gru_units, return_sequences=True, name='activity_encoder')(embedding_activity)
    bn_activity = BatchNormalization(name='bn_activity')(encoded_activity)
    encoded_type = GRU(units=gru_units, return_sequences=True, name='type_encoder')(embedding_type)
    bn_type = BatchNormalization(name='bn_type')(encoded_type)
    encoded_user = GRU(units=gru_units, return_sequences=True, name='user_encoder')(embedding_user)
    bn_user = BatchNormalization(name='bn_user')(encoded_user)

    # Encoder GRU with Batch Normalization for continuous attribute
    encoded_timestamp = GRU(units=gru_units, return_sequences=True, name='timestamp_encoder')(input_timestamp)
    bn_timestamp = BatchNormalization(name='bn_timestamp')(encoded_timestamp)

    # Concatenation of encoded outputs
    concatenated = Concatenate(name='concatenate_encodings')([bn_activity, bn_type, bn_user, bn_timestamp])

    # Decoder GRU
    decoder_output = GRU(units=gru_units, return_sequences=False, name='decoder_gru')(concatenated)
    dropout_layer = Dropout(rate=dropout_rate, name='dropout')(decoder_output)

    # Output layers for predicting the next event's attributes
    output_activity = Dense(num_activities + 1, activation='softmax', name='output_activity')(dropout_layer)
    output_type = Dense(num_types + 1, activation='softmax', name='output_type')(dropout_layer)
    output_user = Dense(num_users + 1, activation='softmax', name='output_user')(dropout_layer)
    output_timestamp = Dense(1, activation='linear', name='output_timestamp')(dropout_layer)  # Linear activation for continuous output

    # Building the model
    model = Model(inputs=[input_activity, input_type, input_user, input_timestamp], outputs=[output_activity, output_type, output_user, output_timestamp])
    model.compile(
        optimizer='adam', 
        loss={
            'output_activity': 'categorical_crossentropy', 
            'output_type': 'categorical_crossentropy', 
            'output_user': 'categorical_crossentropy', 
            'output_timestamp': 'mean_squared_error'
        },
        metrics={
            'output_activity': ['accuracy'], 
            'output_type': ['accuracy'], 
            'output_user': ['accuracy'], 
            'output_timestamp': ['mean_squared_error']
        }
    )

    return model

# Parameters
gru_units = int(2 * E)
num_activities = dataframe_log['activity'].max()
num_types = dataframe_log['type'].max()
num_users = dataframe_log['user'].max()
embedding_dim = 50
dropout_rate = 0.2
model = create_binetv3(num_activities, num_types, num_users, embedding_dim, gru_units, dropout_rate)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 activity_input (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 type_input (InputLayer)     [(None, None)]               0         []                            
                                                                                                  
 user_input (InputLayer)     [(None, None)]               0         []                            
                                                                                                  
 activity_embedding (Embedd  (None, None, 50)             1350      ['activity_input[0][0]']

### Data Splitting

In [67]:
from sklearn.model_selection import train_test_split

# Split the data for each attribute including timestamps
train_activity, test_activity, train_targets_activity, test_targets_activity = train_test_split(
    windows_activity, targets_activity, test_size=0.3, random_state=42)

train_type, test_type, train_targets_type, test_targets_type = train_test_split(
    windows_type, targets_type, test_size=0.3, random_state=42)

train_user, test_user, train_targets_user, test_targets_user = train_test_split(
    windows_user, targets_user, test_size=0.3, random_state=42)

train_timestamp, test_timestamp, train_targets_timestamp, test_targets_timestamp = train_test_split(
    windows_timestamp, targets_timestamp, test_size=0.3, random_state=42)

### Training

In [68]:
from tensorflow.keras.utils import to_categorical

# Convert categorical targets to one-hot encoding
train_targets_activity = to_categorical(train_targets_activity, num_classes=num_activities + 1)
test_targets_activity = to_categorical(test_targets_activity, num_classes=num_activities + 1)

train_targets_type = to_categorical(train_targets_type, num_classes=num_types + 1)
test_targets_type = to_categorical(test_targets_type, num_classes=num_types + 1)

train_targets_user = to_categorical(train_targets_user, num_classes=num_users + 1)
test_targets_user = to_categorical(test_targets_user, num_classes=num_users + 1)

In [69]:
# Train the model
history = model.fit(
    [train_activity, train_type, train_user, train_timestamp],
    [train_targets_activity, train_targets_type, train_targets_user, train_targets_timestamp],
    validation_data=([test_activity, test_type, test_user, test_timestamp], [test_targets_activity, test_targets_type, test_targets_user, test_targets_timestamp]),
    epochs=20,
    batch_size=500
)

Epoch 1/20
41/74 [===============>..............] - ETA: 27s - loss: 8.6905 - output_activity_loss: 2.1524 - output_type_loss: 0.8917 - output_user_loss: 5.2971 - output_timestamp_loss: 0.3493 - output_activity_accuracy: 0.4496 - output_type_accuracy: 0.6743 - output_user_accuracy: 0.1299 - output_timestamp_mean_squared_error: 0.3493

KeyboardInterrupt: 

In [ ]:
# Evaluate the model on the validation set
results = model.evaluate(
    [test_activity, test_type, test_user, test_timestamp],
    [test_targets_activity, test_targets_type, test_targets_user, test_targets_timestamp],
    batch_size=64
)
print(f"Validation Loss: {results[0]}, Validation Accuracy: {results[1]}")


246/246 [==============================] - 13s 52ms/step - loss: 1.1117 - output_activity_loss: 0.2788 - output_type_loss: 0.2228 - output_user_loss: 0.5569 - output_timestamp_loss: 0.0532 - output_activity_accuracy: 0.8825 - output_type_accuracy: 0.9072 - output_user_accuracy: 0.8110 - output_timestamp_mean_squared_error: 0.0532
Validation Loss: 1.1116795539855957, Validation Accuracy: 0.2787676751613617


In [ ]:
# Save the model to an H5 file
model.save('binetv3_mobis.h5')

/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Anomaly Score Computation

- For each event attribute, BINet's softmax layer outputs a probability distribution over possible values
- The anomaly score for a specific attribute value v is calculated by summing all the probabilities from the softmax output that are greater than the probability assigned to v

In [ ]:
# Generate predictions for all inputs
predictions = model.predict([windows_activity, windows_type, windows_user, windows_timestamp])


# Extract predictions for categorical attributes (softmax probabilities)
predictions_activity = predictions[0]
predictions_type = predictions[1]
predictions_user = predictions[2]  
predictions_timestamp = predictions[3]  

1640/1640 [==============================] - 62s 33ms/step


In [ ]:
import numpy as np

def calculate_anomaly_scores(predictions, targets):
    scores = []
    # Loop through each example in the predictions
    for i in range(predictions.shape[0]):
        actual_prob = predictions[i, targets[i]]  # Extract the probability of the true class using target index
        # Calculate anomaly score as sum of probabilities greater than the probability of the actual value
        anomaly_score = np.sum(predictions[i][predictions[i] > actual_prob])
        scores.append(anomaly_score)

    return scores

In [ ]:
anomaly_scores_user = calculate_anomaly_scores(predictions_user, targets_user.astype(int))
anomaly_scores_activity = calculate_anomaly_scores(predictions_activity, targets_activity.astype(int))
anomaly_scores_type = calculate_anomaly_scores(predictions_type, targets_type.astype(int))

In [ ]:
def compute_anomaly_scores_continuous(predictions, actuals):
    # Calculate absolute differences
    differences = np.abs(predictions - actuals)
    
    # Normalize to [0, 1] range
    max_diff = np.max(differences)
    normalized_scores = differences / max_diff if max_diff != 0 else differences
    
    return normalized_scores

In [ ]:
anomaly_scores_timestamp = compute_anomaly_scores_continuous(predictions_timestamp, targets_timestamp)
anomaly_scores_timestamp = anomaly_scores_timestamp.flatten()
anomaly_scores_timestamp = anomaly_scores_timestamp.tolist()

### Threshold (lowest plateau)

In [ ]:
import numpy as np

def calculate_anomaly_ratio(scores, threshold):
    """
    Calculate the anomaly ratio for a given threshold.
    """
    return np.mean(scores > threshold)

def find_plateaus(scores, epsilon=1e-4, min_plateau_length=10):
    """
    Identify the lowest plateau in the anomaly ratio function and calculate the mean-centered threshold.
    """
    scores = np.array(scores)  # Convert scores to a NumPy array
    sorted_scores = np.sort(scores)
    
    # Remove duplicate values
    unique_thresholds, unique_indices = np.unique(sorted_scores, return_index=True)
    anomaly_ratios = np.array([calculate_anomaly_ratio(scores, t) for t in unique_thresholds])
    
    # Calculate first and second derivatives
    first_derivatives = np.diff(anomaly_ratios) / np.diff(unique_thresholds)
    second_derivatives = np.diff(first_derivatives) / np.diff(unique_thresholds[:-1])
    
    # Identify plateaus where the first derivative is close to zero
    plateau_indices = np.where(np.abs(first_derivatives) < epsilon)[0]
    
    # Group consecutive indices to identify continuous plateaus
    grouped_plateaus = np.split(plateau_indices, np.where(np.diff(plateau_indices) != 1)[0] + 1)
    
    # Filter plateaus based on minimum length
    long_plateaus = [g for g in grouped_plateaus if len(g) >= min_plateau_length]
    
    if long_plateaus:
        # Take the first long plateau and find the mean threshold in this plateau
        first_plateau = long_plateaus[0]
        plateau_thresholds = unique_thresholds[first_plateau]
        return np.mean(plateau_thresholds)
    else:
        # If no plateau is found, return a default value, e.g., the 90th percentile
        percentile_90 = np.percentile(sorted_scores, 90)
        if percentile_90 == 1.0:
            return 0.4
        else:
            return percentile_90

In [ ]:
threshold_activity = find_plateaus(anomaly_scores_activity)
threshold_user = find_plateaus(anomaly_scores_user)
threshold_type = find_plateaus(anomaly_scores_type)
threshold_timestamp = find_plateaus(anomaly_scores_timestamp)

### Detection

In [ ]:
def detect_anomalies(anomaly_scores, threshold):
    labels = [1 if score > threshold else 0 for score in anomaly_scores]
    return labels

In [ ]:
# Detect anomalies based on the calculated anomaly scores and thresholds
labels_activity = detect_anomalies(anomaly_scores_activity, threshold_activity)
labels_user = detect_anomalies(anomaly_scores_user, threshold_user)
labels_type = detect_anomalies(anomaly_scores_type, threshold_type)
labels_timestamp = detect_anomalies(anomaly_scores_timestamp, threshold_timestamp)

# Mapping

In [ ]:
import pandas as pd

# Create a DataFrame from the case_indices_array corresponding to case_resource
mapping = pd.DataFrame({'case': case_indices})
mapping['predicted_activity'] = labels_activity
mapping['predicted_user'] = labels_user
mapping['predicted_type'] = labels_type
mapping['predicted_timestamp'] = labels_timestamp

mapping

,case,predicted_activity,predicted_user,predicted_type,predicted_timestamp
0,0,0,1,1,0
1,0,0,1,1,0
2,0,0,0,0,0
3,0,0,0,0,1
4,0,0,0,0,0
...,...,...,...,...,...
52450,3353,0,0,0,0
52451,3353,0,0,0,0
52452,3353,0,0,0,0
52453,3353,0,0,0,1


In [ ]:
# Create a boolean DataFrame where each value is True if the value is 1
contains_one = (mapping[['predicted_activity', 'predicted_user', 'predicted_type', 'predicted_timestamp']] == 1)

# Group by 'case' and check if there's at least one 'True' in any of the columns
case_prediction = contains_one.groupby(mapping['case']).any().any(axis=1)
case_prediction

case
0       True
1       True
2       True
3       True
4       True
        ... 
3349    True
3350    True
3351    True
3352    True
3353    True
Length: 3354, dtype: bool

In [ ]:
# Create a boolean DataFrame where each value is True if the value is 1
contains_one = (mapping[['predicted_activity']] == 1)

# Group by 'case' and check if there's at least one 'True' in any of the columns
case_prediction = contains_one.groupby(mapping['case']).any().any(axis=1)
case_prediction

case
0       False
1        True
2       False
3        True
4        True
        ...  
3349     True
3350     True
3351     True
3352     True
3353     True
Length: 3354, dtype: bool

# Ground Truth

In [ ]:
def generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking):
    from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments
    from pm4py.algo.conformance.alignments.petri_net import variants
    from pm4py.objects.petri_net.utils import align_utils
    max_events=0
    for trace in log:
        counter=0
        for event in trace:
            counter+=1
        if counter > max_events:
            max_events=counter
    parameters={}
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_SYNC_COST_FUNCTION] = list(map(lambda i: .1*i, range(max_events*2)))
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_TRACE_COST_FUNCTION]=list(map(lambda i: align_utils.STD_MODEL_LOG_MOVE_COST-.1*i, range(max_events*2)))
    aligned_traces = alignments.apply_log(log, net, initial_marking, final_marking, variant=variants.state_equation_a_star, parameters=parameters)
    return aligned_traces

In [ ]:
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.bpmn.importer import importer as bpmn_importer
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments_petri

# 2. Import the given BPMN model
bpmn_graph = bpmn_importer.apply("../../data/model/MobisToBe.bpmn")

# 3. Convert the BPMN to a Petri net
net, initial_marking, final_marking = pm4py.convert_to_petri_net(bpmn_graph)

aligned_traces = generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking)

aligning log, completed variants ::   0%|          | 0/295 [00:00<?, ?it/s]

In [ ]:
def extract_conformance_status_by_fitness(aligned_traces):
    conformance_status = []
    for alignment in aligned_traces:
        fitness = alignment['fitness']
        # If the fitness is 1.0, the trace is conforming
        if fitness == 1.0:
            conformance_status.append(1)
        else:
            conformance_status.append(0)
    return conformance_status

# Get the conformance status list from the aligned traces
conformance = extract_conformance_status_by_fitness(aligned_traces)

In [ ]:
ground_truth = pd.DataFrame({'conformity': conformance})
ground_truth['predicted'] = case_prediction

# Convert False to 0 and True to 1
ground_truth['predicted'] = [int(value) for value in ground_truth['predicted']]
ground_truth['predicted'] = 1 - ground_truth['predicted']
ground_truth

,conformity,predicted
0,1,0
1,0,0
2,1,0
3,0,0
4,0,0
...,...,...
3349,1,0
3350,0,0
3351,1,0
3352,1,0


# Evaluation

In [ ]:
# Calculating TP, TN, FP, FN
TP = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 1)).sum()
TN = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 0)).sum()
FP = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 1)).sum()
FN = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 0)).sum()

In [ ]:
# Calculate accuracy
accuracy = (TP + TN) / (TP + TN + FP + FN)
print(f"Accuracy: {accuracy:.3f}")

Accuracy: 0.511


In [ ]:
# Calculate f1

precision = TP / (TP + FP)
recall = TP / (TP + FN)

f1 = 2 * ((precision * recall) / (precision + recall))
print(f"F1: {f1:.3f}")

F1: 0.057


### Dev (Non Conform Traces)

In [ ]:
# Calculate precision for Dev
precision = TN / (TN + FN)
print(f"Precision: {precision:.3f}")

Precision: 0.504


In [ ]:
# Calculate recall for Dev
recall = TN / (TN + FP)
print(f"Recall: {recall:.3f}")

Recall: 1.000


### No Dev (Conform Traces)

In [ ]:
# Calculate precision for No Dev
precision = TP / (TP + FP)
print(f"Precision: {precision:.3f}")

Precision: 1.000


In [ ]:
# Calculate recall for No Dev
recall = TP / (TP + FN)
print(f"Recall: {recall:.3f}")

Recall: 0.030


### AUC-ROC

In [ ]:
import pandas as pd
from sklearn.metrics import roc_auc_score

# Assuming ground_truth is your DataFrame
# Make sure 'conformity' contains actual labels (0 or 1)
# and 'predicted' contains predicted probabilities or scores
auc_roc = roc_auc_score(ground_truth['conformity'], ground_truth['predicted'])
auc_roc

0.514792899408284